In [4]:
#import libraries
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [5]:
#load the dataset into a pandas dataframe
file_path = os.path.join('data', 'Booking_Data_Cleaned.csv')
df = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Booking_Data_Cleaned.csv'

In [4]:
df.head()

,Hotel,Booking Date,Arrival Date,Lead Time,Nights,Guests,Distribution Channel,Customer Type,Country,Deposit Type,...,Stay Duration Category,Revenue Category,Revenue Loss Category,ADR Category,Booking Month,Booking Day,Arrival Month,Arrival Day,Booking Year,Arrival Year
0,Resort,2014-07-24,2015-07-01,342,0,2,Direct,Transient,Portugal,No Deposit,...,short,low,low,low,July,Thursday,July,Wednesday,2014,2015
1,Resort,2015-06-24,2015-07-01,7,1,1,Direct,Transient,United Kingdom,No Deposit,...,short,low,low,low,June,Wednesday,July,Wednesday,2015,2015
2,Resort,2015-06-18,2015-07-01,13,1,1,Corporate,Transient,United Kingdom,No Deposit,...,short,low,low,low,June,Thursday,July,Wednesday,2015,2015
3,Resort,2015-06-17,2015-07-01,14,2,2,Online Travel Agent,Transient,United Kingdom,No Deposit,...,short,medium,low,low,June,Wednesday,July,Wednesday,2015,2015
4,Resort,2015-06-17,2015-07-01,14,2,2,Online Travel Agent,Transient,United Kingdom,No Deposit,...,short,medium,low,low,June,Wednesday,July,Wednesday,2015,2015


We know that the dataset very imabalanced from univariate analysis of Cancelled column. Lets balance the dataset using SMOTE method

In [ ]:

# Identify categorical and numerical columns
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X_train.select_dtypes(exclude=['object', 'category']).columns.tolist()

# Create a column transformer with appropriate transformers for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Fit the preprocessor and transform the training data
X_train_processed = preprocessor.fit_transform(X_train)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit SMOTE to the training data
X_train_smote, y_train_smote = smote.fit_resample(X_train_processed, y_train)

# Checking the balance after SMOTE
print(f"Before SMOTE, counts of label '1': {sum(y_train == 1)}")
print(f"Before SMOTE, counts of label '0': {sum(y_train == 0)}\n")
print(f"After SMOTE, counts of label '1': {sum(y_train_smote == 1)}")
print(f"After SMOTE, counts of label '0': {sum(y_train_smote == 0)}")


In [7]:
# Separating features and target variable
X = df.drop(['Cancelled (0/1)'], axis=1)
y = df['Cancelled (0/1)']

# Identifying categorical columns for one-hot encoding
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# Creating a preprocessing pipeline for automatic one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Passthrough numerical columns as they are
)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Logistic Regression Model
lr_pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=1000))
lr_pipeline.fit(X_train, y_train)

# Random Forest Model
rf_pipeline = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100))
rf_pipeline.fit(X_train, y_train)

# Model Evaluation
for model, name in zip([lr_pipeline, rf_pipeline], ['Logistic Regression', 'Random Forest']):
    y_pred = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(f"{name} Precision: {precision_score(y_test, y_pred):.2f}")
    print(f"{name} Recall: {recall_score(y_test, y_pred):.2f}")
    print(f"{name} F1-Score: {f1_score(y_test, y_pred):.2f}")
    print(f"{name} ROC-AUC: {roc_auc_score(y_test, y_pred):.2f}\n")


c:\Users\1ga17\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 1.00
Logistic Regression Precision: 1.00
Logistic Regression Recall: 1.00
Logistic Regression F1-Score: 1.00
Logistic Regression ROC-AUC: 1.00

Random Forest Accuracy: 1.00
Random Forest Precision: 1.00
Random Forest Recall: 1.00
Random Forest F1-Score: 1.00
Random Forest ROC-AUC: 1.00

